Author: Ernest Mack

Comments: This could be done more elegently using other methods however, this is intended as a brute-force exercise using bare-bones coding to get the job done.

### This is a brute-force web scraper using python, requests and beautifulsoup to get the lifting done.

The skills developed use requests to to access the web site and get the html and then I use beautiful soup to parse the html tags for the data I want to mine then I convert the data to various lists from which pandas dataframes are created. Then I concatenate the dataframes to create a single dataframe with multiple columns of data. I know that was one heck of a run-on sentence. Anyway, as an exercise I then write-out the dataframe to a .CSV for sharing if I want. now I can focus on doing some data visualizations and manipulation.

### View the properly rendered verion here:

https://nbviewer.jupyter.org/github/SeriousHyena/Sandbox/blob/master/a_simple_scraper.ipynb

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### Use Python Requests to get connected to the site and get the html

In [2]:
r = requests.get('https://www.jdpower.com/Cars/Consumer-Reviews')

In [3]:
url = 'https://www.jdpower.com/Cars/Consumer-Reviews'
r = requests.get(url)
content = r.text

### Use BeautifulSoup to parse the html

https://www.crummy.com/software/BeautifulSoup/bs4/doc/


### First inspect the web page under consideration.

I do this by navigating out to the web page and then use the 'inspect element' feature of firefox. This allows me to determine the tag that references the data I am considering mining.

### Create a beuatifulsoup element to hold the parsed html

In [4]:
soup = BeautifulSoup(content, 'html.parser')

### Now start parsing the html

In [5]:
cars = soup.find_all('section', 'site-content')

### Iterate through the html tags.

### A function to iterate the various html tags

In [6]:
tag_item = [] #a list for the descriptions I will call this function as required based on the html
              #tags and the data I want to extract  
def get_tags(tag_name, tag_desc): 
    tag_item.clear() #clear the list contentas for re-use
    for paragraph in soup.find_all(tag_name,tag_desc):
        tag_item.append(str(paragraph.text))

    

### create lists to store the tag data I want to mine

In [7]:
get_tags('div', 'title heading-s')

In [8]:
#here I have to use the python copy() method otherwise I will not be able to save new list contents
car_name = tag_item.copy()

In [9]:
car_name[:5]

['2019 Mercedes-Benz GLC',
 '2019 Jeep Cherokee',
 '2019 Mercedes-Benz GLA',
 '2019 Chevrolet Tahoe',
 '2019 Audi Q7']

In [10]:
get_tags('div', 'heading-l')
rating = tag_item.copy()
rating[:12]

['4', '5', '5', '5', '5', '4', '5', '5', '4', '4', '3', '4']

In [11]:
get_tags('div', 'review-body-text')
review = tag_item.copy()
review[:2]

['THE MERCEDES BENZ 2019 GLC IS VERY SMART. SOMETIMES IT IS A SMART ALEC!  SO MUCH TECHNOLOGY THAT IT IS DIFFICULT TO FEEL COMFORTABLE WITH ALL THE BELLS AND WHISTLES. BE PREPARED TO GET ALL THE LATEST IN TECHNOLOGY WHETHER YOU WANT IT OR NOT.',
 'Excellent, reliable vehicle that handles the road like a pro. Perfect road trip and everyday car for my growing family.']

### This section of code gets the owner verified review info

In [12]:
get_tags('div', 'author')
author = tag_item.copy()
author[:5]

['JILL B', 'Sarah', 'ROCHELLE', 'Anonymous', 'Anonymous']

In [13]:
get_tags('div', 'author-attributes-wrapper')
author_attrib = tag_item.copy()
author_attrib[:5]

['\nApr 02, 2019\n|\n500 miles\n|\nOwned 7 months\n',
 '\nApr 02, 2019\n',
 '\nApr 02, 2019\n|\n5,600 miles\n|\nOwned 11 months\n',
 '\nApr 02, 2019\n|\n9,000 miles\n|\nOwned 6 months\n',
 '\nApr 01, 2019\n']

### Remove unwanted characters from the author_attrib column

In [45]:
author_attrib = [i.replace('\n', '') for i in author_attrib]
author_attrib = [i.replace('|', '') for i in author_attrib]

In [51]:
author_attrib[:5] #verify my string cleaning worked

['Apr 02, 2019500 milesOwned 7 months',
 'Apr 02, 2019',
 'Apr 02, 20195,600 milesOwned 11 months',
 'Apr 02, 20199,000 milesOwned 6 months',
 'Apr 01, 2019']

### Now create a Pandas dataframe

### First I will concatenate the two dataframes so I have the description and price on one row.

I have concatenated several dataframes here so will be looking for a better way to do this however, this works fine but I think I could do a more elegant coding job.

In [46]:
df1 = pd.DataFrame(car_name)
df2 = pd.DataFrame(rating)
df_review = pd.DataFrame(review)
df_author = pd.DataFrame(author)
df_author_attrib = pd.DataFrame(author_attrib)

In [42]:
df3 = pd.concat([df1, df2, df_review, df_author, df_author_attrib], axis=1)

In [43]:
df3.columns = ['Car Name', 'Car Rating', 'Review', 'Author', 'Author Info']

In [47]:
df3.head()

,Car Name,Car Rating,Review,Author,Author Info
0,2019 Mercedes-Benz GLC,4,THE MERCEDES BENZ 2019 GLC IS VERY SMART. SOME...,JILL B,"Apr 02, 2019500 milesOwned 7 months"
1,2019 Jeep Cherokee,5,"Excellent, reliable vehicle that handles the r...",Sarah,"Apr 02, 2019"
2,2019 Mercedes-Benz GLA,5,I LOVE THE WAY MY CAR ACCELARATES. I LOVE MY W...,ROCHELLE,"Apr 02, 20195,600 milesOwned 11 months"
3,2019 Chevrolet Tahoe,5,The Chevy Tahoe is very versatile. Seats lay ...,Anonymous,"Apr 02, 20199,000 milesOwned 6 months"
4,2019 Audi Q7,5,"Great vehicle, great performance. Very impres...",Anonymous,"Apr 01, 2019"


### Save the file to a .CSV for storage and sharing.

In [49]:
#this will show a file access error on github of course but does work locally as long as the file is not
#opened in excel.
df3.to_csv('car_reviews.csv')

### Next we can do some data visualization and manipulation